In [1]:
import pandas as pd
import json
from pathlib import Path
from typing import List
from functools import reduce
import numpy as np
from sklearn.metrics import f1_score
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from xgboost import XGBClassifier


In [15]:
from sklearn import metrics

In [2]:
import joblib

In [42]:
def ensemble(true: np.array, preds: np.array, strategy='OR'):
    final = []
    clf = None
    
    if strategy == 'OR':
        for pred in preds:
            final.append(1 if any(pred) else 0)
    
    elif strategy == 'AND':
        for pred in preds:
            final.append(1 if all(pred) else 0)
            
    elif strategy == 'majority':
        for pred in preds:
            final.append(1 if sum(pred)>(len(pred)/2) else 0)
            
    elif strategy == 'blend_rf':
        clf = RandomForestClassifier(max_depth=100, 
                                     n_estimators=3, 
                                     random_state=0)
        clf.fit(preds, true.ravel())
        final = clf.predict(preds)
    elif strategy == 'blend_xgb':
        clf = XGBClassifier(n_estimators=1000,
                                       max_depth=5,
                            learning_rate=0.1,
                                       verbosity=1)
        clf.fit(preds, true.ravel())
        final = clf.predict(preds)

    return final, clf

In [4]:
def merge_cnn_preds_with_df(cnn_preds_fp, dev, test):
    pred_df_dev = dev.copy()
    pred_df_test = test.copy()
    with open(cnn_preds_fp[0], 'rb') as handle:
        pred_df_dev['Prediction'] = pickle.load(handle)
        pred_df_dev['Prediction'] = pred_df_dev['Prediction'].astype(int)
    with open(cnn_preds_fp[1], 'rb') as handle:
        pred_df_test['Prediction'] = pickle.load(handle)
        pred_df_test['Prediction'] = pred_df_test['Prediction'].astype(int)
    return pred_df_dev, pred_df_test

In [5]:
test = pd.read_csv(Path('/media/sarthak/HDD/data_science/fnp_resources/data/task1/all_combined/test.csv'))
dev = pd.read_csv(Path('/media/sarthak/HDD/data_science/fnp_resources/data/task1/all_combined/dev.csv'))

In [28]:
# code for transformers prediction
fnp_pred_dfs_dev = []
fnp_pred_dfs_test = []

file_paths = [[Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/57_1/output/best_model/inference_dev/predictions.csv'),
             Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/57_1/output/best_model/inference/predictions.csv')],
    [Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/64_1/output/best_model/inference_dev/predictions.csv'),
             Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/64_1/output/best_model/inference/predictions.csv')],
              [Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/67_1/output/best_model/inference_dev/predictions.csv'),
             Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/67_1/output/best_model/inference/predictions.csv')],
              [Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/69_1/output/best_model/inference_dev/predictions.csv'),
             Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/69_1/output/best_model/inference/predictions.csv')]]
"""
file_paths = [ [Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/61_1/output/best_model/inference_dev/predictions.csv'),
             Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/61_1/output/best_model/inference/predictions.csv')],
    [Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/62_1/output/best_model/inference_dev/predictions.csv'),
             Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/62_1/output/best_model/inference/predictions.csv')],
    
    [Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/64_1/output/best_model/inference_dev/predictions.csv'),
             Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/64_1/output/best_model/inference/predictions.csv')],
    [Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/67_1/output/best_model/inference_dev/predictions.csv'),
             Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/67_1/output/best_model/inference/predictions.csv')],
              [Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/69_1/output/best_model/inference_dev/predictions.csv'),
             Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/69_1/output/best_model/inference/predictions.csv')],
             [Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/87_1/output/best_model/inference_dev/predictions.csv'),
             Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/87_1/output/best_model/inference/predictions.csv')]]
"""
for fp in file_paths:
    fnp_pred_dfs_dev.append(pd.read_csv(fp[0]))
    fnp_pred_dfs_test.append(pd.read_csv(fp[1]))

In [29]:
# code for CNN predictions
cnn_pred_dfs_dev = []
cnn_pred_dfs_test = []
"""
file_paths = [['/media/sarthak/HDD/TUM/Thesis/thesis-sarthak/src/tc/experiments/fincausal_all_combined/preds_val.pkl', 
               '/media/sarthak/HDD/TUM/Thesis/thesis-sarthak/src/tc/experiments/fincausal_all_combined/preds.pkl']]
"""
file_paths = [['/media/sarthak/HDD/TUM/Thesis/thesis-sarthak/src/tc/experiments/fincausal_all_combined/preds_val.pkl', 
               '/media/sarthak/HDD/TUM/Thesis/thesis-sarthak/src/tc/experiments/fincausal_all_combined/preds.pkl']]

for fp in file_paths:
    cnn_pred_df_dev, cnn_pred_df_test = merge_cnn_preds_with_df(fp, dev, test)
    cnn_pred_dfs_dev.append(cnn_pred_df_dev)
    cnn_pred_dfs_test.append(cnn_pred_df_test)

In [30]:
data_frames_test = fnp_pred_dfs_test
data_frames = fnp_pred_dfs_dev
k = np.arange(len(data_frames)).astype(str)
df_merged = pd.concat([x.set_index('unique_id') for x in data_frames], axis=1, join='inner', keys=k)
df_merged_test = pd.concat([x.set_index('unique_id') for x in data_frames_test], axis=1, join='inner', keys=k)
df_merged.columns = df_merged.columns.map('_'.join)
df_merged_test.columns = df_merged_test.columns.map('_'.join)

cols_gold = ['0_Gold'] 
cols_pred = [i+'_Prediction' for i in k]
df_merged = df_merged[cols_gold + cols_pred]
df_merged_test = df_merged_test[cols_gold + cols_pred]

In [31]:
test.head()

,Index,Text,Gold,unique_id
0,352.00039,One bit of advice Orton had for young finance ...,0,f56182ea-5095-4934-889c-f18c1a26134c
1,8.00001,President Muhammadu Buhari has disclosed that ...,1,06cc12f1-1df2-43ef-898a-85ebb29bdede
2,53.00010,Advent of compressed exhaust systems based on ...,0,a2705c77-271b-4870-b948-8b688657f39c
3,533.00022,Catholic leaders have also suggested the UK go...,0,7966c427-9695-4e2f-a098-a5d385d6e0c4
4,316.00040,(NasdaqGS:CME) is 66. A company with a value o...,0,14e3e218-9f6b-4949-96c7-62ca68598d55


In [32]:
df_merged.head()

,0_Gold,0_Prediction,1_Prediction,2_Prediction,3_Prediction
unique_id,,,,,
53f913fa-0516-4f52-9726-d204bb60c848,0,0,0,0,0
e105db26-3a32-404a-873d-797f33e5ed98,0,0,0,0,0
f5849841-7bbd-4f58-afae-096701373aaa,0,0,0,0,0
76751223-e05b-41e0-a1f5-8dedc52ad7b1,0,0,0,0,0
4f94535d-7c7f-4d9a-ab9a-ff8b3099b350,0,0,0,0,0


In [33]:
df_merged_test.head()

,0_Gold,0_Prediction,1_Prediction,2_Prediction,3_Prediction
unique_id,,,,,
f56182ea-5095-4934-889c-f18c1a26134c,0,0,0,0,0
06cc12f1-1df2-43ef-898a-85ebb29bdede,1,0,0,0,0
a2705c77-271b-4870-b948-8b688657f39c,0,0,0,0,0
7966c427-9695-4e2f-a098-a5d385d6e0c4,0,0,0,0,0
14e3e218-9f6b-4949-96c7-62ca68598d55,0,0,0,0,0


In [38]:
df_merged['Prediction'], meta_clf = ensemble(true=df_merged[cols_gold].values, 
                                             preds=df_merged[cols_pred].values,
                                             strategy='blend_xgb')
df_merged_test['Prediction'] = meta_clf.predict(df_merged_test[cols_pred].values)

print('val: ', metrics.precision_recall_fscore_support(df_merged['0_Gold'].tolist(), df_merged['Prediction'], average='weighted'))
print('test: ', metrics.precision_recall_fscore_support(df_merged_test['0_Gold'].tolist(), df_merged_test['Prediction'], average='weighted'))

val:  (0.9647423172856094, 0.9657430730478589, 0.9651642898889585, None)
test:  (0.9582267824413016, 0.9605621033544878, 0.9589876604066631, None)


In [174]:
joblib.dump(meta_clf, Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/submissions/metaclf_bertsandcnnandrules.joblib'))

['/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/submissions/metaclf_bertsandcnnandrules.joblib']

In [43]:
df_merged['Prediction'], meta_clf = ensemble(true=df_merged[cols_gold].values, 
                                             preds=df_merged[cols_pred].values,
                                             strategy='majority')
df_merged_test['Prediction'], meta_clf = ensemble(true=df_merged_test[cols_gold].values, 
                                             preds=df_merged_test[cols_pred].values,
                                             strategy='majority')

print('val: ', metrics.precision_recall_fscore_support(df_merged['0_Gold'].tolist(), df_merged['Prediction'], average='weighted'))
print('test: ', metrics.precision_recall_fscore_support(df_merged_test['0_Gold'].tolist(), df_merged_test['Prediction'], average='weighted'))

val:  (0.9613595244573224, 0.9632241813602015, 0.9620000999270045, None)
test:  (0.9557739530920735, 0.9592021758839528, 0.9554399324236738, None)


In [236]:
meta_clf.feature_importances_, 

(array([0.02110144, 0.6654096 , 0.3067513 , 0.00673764], dtype=float32),)

In [361]:
# generate final vals and tests
ensembled_dev = dev.set_index('unique_id').join(df_merged[['Prediction']])
# print(f1_score(ensembled_dev['Gold'].tolist(), ensembled_dev['Prediction'], average='weighted'))
ensembled_test = test.set_index('unique_id').join(df_merged_test[['Prediction']])
# print(f1_score(ensembled_test['Gold'].tolist(), ensembled_test['Prediction'], average='weighted'))

In [363]:
ensembled_dev.reset_index(level=0, inplace=False).to_csv('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/74_1/output/best_model/inference_dev/predictions.csv', index=False)
ensembled_test.reset_index(level=0, inplace=False).to_csv('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/74_1/output/best_model/inference/predictions.csv', index=False)

# Generate Prediction

In [163]:
predict_df_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/task1/practice/all.csv')
predict_df = pd.read_csv(predict_df_path)
print('shape of data is: {}'.format(predict_df.shape))
predict_df.head()

shape of data is: (13547, 4)


,Index,Text,Gold,unique_id
0,1.00001,Florida raking in billions as Americans aband...,0,94bdc788-8aaa-49c1-9473-8222392e0c6e
1,1.00002,"Recently, changes to the U.S. tax code have e...",0,0af1d3f6-6aa9-461c-b921-0f8b90c83328
2,1.00003,"MORE FROM FOXBUSINESS.COM... As it turns out,...",0,5166917b-fe82-4e82-ba73-c67db70d957b
3,1.00004,"According to a new study from LendingTree, wh...",0,a52522d7-e25e-4b43-a510-44446d500443
4,1.00005,The Sunshine State drew in a net influx of ab...,1,517e2ae2-7d16-40af-a2a7-eb567841f9f8


In [165]:
predict_df['Prediction'] = merged['ensemble_preds'].tolist()

In [166]:
predict_df

,Index,Text,Gold,unique_id,Prediction
0,1.00001,Florida raking in billions as Americans aband...,0,94bdc788-8aaa-49c1-9473-8222392e0c6e,0
1,1.00002,"Recently, changes to the U.S. tax code have e...",0,0af1d3f6-6aa9-461c-b921-0f8b90c83328,0
2,1.00003,"MORE FROM FOXBUSINESS.COM... As it turns out,...",0,5166917b-fe82-4e82-ba73-c67db70d957b,0
3,1.00004,"According to a new study from LendingTree, wh...",0,a52522d7-e25e-4b43-a510-44446d500443,0
4,1.00005,The Sunshine State drew in a net influx of ab...,1,517e2ae2-7d16-40af-a2a7-eb567841f9f8,1
...,...,...,...,...,...
13542,590.00061,Contact transpo@gmu.edu with questions.,0,0294a30b-c9a5-4f89-ad3e-1188daa17543,0
13543,590.00062,Campus Fire Safety Month September is Campus...,0,77c8d28b-f1cf-4d04-b1fe-5ce06408d212,0
13544,590.00063,"Review the university's Fire Safety Plan, whi...",0,2e56e743-d171-43fd-a18c-1b4a4eca2aae,0
13545,590.00064,Contact Meredith Muckerman at 703-993-9715 or...,0,c1357ffa-d742-4f14-9f29-1d76808ba3c1,0


In [181]:
predict_df[['Index', 'Text', 'Prediction']].to_csv(Path('/media/sarthak/HDD/data_science/fnp_resources/fincausal_t1_models/37/output/best_model/inference/predictions.csv'), index=False, sep=';')